In [1]:
45

45

In [2]:
import os
from dotenv import load_dotenv

import pandas as pd
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

c:\MSc.FY Projects\anime\anienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
GROQ_API_KEY =os.getenv("GROQ_API_KEY ")
os.getenv("GROQ_API_KEY ") = GROQ_API_KEY


In [3]:
import os
from langchain.chat_models import init_chat_model

model = init_chat_model("groq:llama-3.3-70b-versatile")
response = model.invoke("Why am i studying at DOT pune")
response

AIMessage(content="It seems like you're studying at DOT (Department of Technology) in Pune, but without more context, it's difficult for me to provide a specific reason why you're studying there.\n\nHowever, I can try to provide some general information about DOT Pune. The Department of Technology is a part of the Savitribai Phule Pune University (SPPU) and is known for its academic programs in various fields of technology, such as engineering, computer science, and information technology.\n\nSome possible reasons why you might be studying at DOT Pune include:\n\n1. **Academic reputation**: DOT Pune has a good reputation for providing quality education in the field of technology.\n2. **Course offerings**: DOT Pune offers a range of undergraduate and postgraduate programs in various fields of technology, which might align with your interests and career goals.\n3. **Research opportunities**: As a research-oriented institution, DOT Pune provides opportunities for students to engage in res

DATA LOADING CLASS

In [12]:
class AnimeDataLoader:
    def __init__(self, original_csv: str, processed_csv: str):
        self.original_csv = original_csv
        self.processed_csv = processed_csv

    def load_and_process(self):
        df = pd.read_csv(
            self.original_csv,
            encoding="utf-8",
            on_bad_lines="skip"
        ).dropna()

        required_cols = {"Name", "Genres", "sypnopsis"}
        if not required_cols.issubset(df.columns):
            raise ValueError("Missing required columns in CSV")

        df["combined_info"] = (
            "Title: " + df["Name"]
            + " Overview: " + df["sypnopsis"]
            + " Genres: " + df["Genres"]
        )

        df[["combined_info"]].to_csv(
            self.processed_csv,
            index=False,
            encoding="utf-8"
        )

        return self.processed_csv


In [13]:
original_csv =  "data/anime_with_synopsis.csv"
processed_csv = "data/anime_updated.csv"

data_loader = AnimeDataLoader(original_csv, processed_csv)

data_loader.load_and_process()


'data/anime_updated.csv'

DATA SPLITTING


In [9]:
class VectorStoreBuilder:
    def __init__(self, csv_path: str, persist_dir: str = "chroma_db"):
        self.csv_path = csv_path
        self.persist_dir = persist_dir
        self.embedding = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2"
        )

    def build_and_save_vectorstore(self):
        loader = CSVLoader(
            file_path=self.csv_path,
            encoding="utf-8",
            metadata_columns=[]
        )

        documents = loader.load()

        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=0
        )
        chunks = splitter.split_documents(documents)

        db = Chroma.from_documents(
            chunks,
            self.embedding,
            persist_directory=self.persist_dir
        )
        db.persist()

    def load_vector_store(self):
        return Chroma(
            persist_directory=self.persist_dir,
            embedding_function=self.embedding
        )


In [14]:
print("Building vector database...")

original_csv = "data/anime_with_synopsis.csv"
processed_csv = "data/anime_updated.csv"
persist_dir = "chroma_db"

loader = AnimeDataLoader(original_csv, processed_csv)
processed_csv_path = loader.load_and_process()

vector_builder = VectorStoreBuilder(
    processed_csv_path,
    persist_dir=persist_dir
)
vector_builder.build_and_save_vectorstore()

print("Vector database ready.")


Building vector database...


c:\MSc.FY Projects\anime\anienv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regu

Vector database ready.


C:\Users\hp\AppData\Local\Temp\ipykernel_3672\3258009850.py:29: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [15]:
print(" STILL BUILDING A VECTOR STORE ")

vector_builder = VectorStoreBuilder(
    csv_path=processed_csv_path,
    persist_dir=persist_dir
)

vector_builder.build_and_save_vectorstore()

print("Vector database ready")

'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: c3f54b67-3d5f-4895-adb6-f81f2fcfc1f7)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


 STILL BUILDING A VECTOR STORE 


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 1e7a7e94-03e9-4ba0-bd2e-0e44b0372522)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: f19e3d94-d10e-4a72-ad34-1b1c932a85ec)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-M

Vector database ready


RETRIVER TOOL

In [ ]:
vector_builder = VectorStoreBuilder(
    csv_path="",
    persist_dir="chroma_db"
)
retriver = vector_builder.load_vector_store().as_retriever()

'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 093e0cf2-9616-4b1c-95e3-fe51d505c2b0)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 3846e3e3-40bc-478a-9bde-e9d91bbe7979)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-M

In [20]:
query = "tell me about naruto"

retriver.invoke(query)

[Document(metadata={'row': 10, 'source': 'data/anime_updated.csv'}, page_content="combined_info: Title: Naruto Overview: oments prior to Naruto Uzumaki's birth, a huge demon known as the Kyuubi, the Nine-Tailed Fox, attacked Konohagakure, the Hidden Leaf Village, and wreaked havoc. In order to put an end to the Kyuubi's rampage, the leader of the village, the Fourth Hokage, sacrificed his life and sealed the monstrous beast inside the newborn Naruto. Now, Naruto is a hyperactive and knuckle-headed ninja still living in Konohagakure. Shunned because of the Kyuubi inside him, Naruto struggles to find his place in the village, while his burning desire to become the Hokage of Konohagakure leads him not only to some great new friends, but also some deadly foes. Genres: Action, Adventure, Comedy, Super Power, Martial Arts, Shounen"),
 Document(metadata={'source': 'data/anime_updated.csv', 'row': 10}, page_content="combined_info: Title: Naruto Overview: oments prior to Naruto Uzumaki's birth,

In [22]:
@tool
def anime_retriever_tool(query: str) -> str:
    """
    Use this tool to search the anime knowledge base.

    Always call this tool for anime-related questions such as:
    recommendations, similarity search, genres, or plot summaries.

    Input:
    - query: User's anime preference or question.

    Output:
    - Relevant anime information retrieved from the vector database.
    """
    docs = retriver.invoke(query)
    return "\n\n".join(doc.page_content for doc in docs)


In [23]:
model_with_tools = model.bind_tools([anime_retriever_tool])

In [30]:
messages = [
    {
        "role":"user",
        "content":"can you suggest something similar to naruto with a joke"
    }
]

In [25]:
ai_msg =model_with_tools.invoke(messages)
messages.append(ai_msg)


In [26]:
messages

[{'role': 'user', 'content': 'can you suggest something similar to naruto'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'hng4af11n', 'function': {'arguments': '{"query":"anime similar to Naruto"}', 'name': 'anime_retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 335, 'total_tokens': 356, 'completion_time': 0.054485191, 'completion_tokens_details': None, 'prompt_time': 0.018389899, 'prompt_tokens_details': None, 'queue_time': 0.053136704, 'total_time': 0.07287509}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c1354-3f4e-7eb3-8895-ef72ca1f4adb-0', tool_calls=[{'name': 'anime_retriever_tool', 'args': {'query': 'anime similar to Naruto'}, 'id': 'hng4af11n', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 335, 'output_tok

In [27]:
for tool_call in ai_msg.tool_calls:
    tool_result = anime_retriever_tool.invoke(tool_call)
    messages.append(tool_result)

In [28]:
messages

[{'role': 'user', 'content': 'can you suggest something similar to naruto'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'hng4af11n', 'function': {'arguments': '{"query":"anime similar to Naruto"}', 'name': 'anime_retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 335, 'total_tokens': 356, 'completion_time': 0.054485191, 'completion_tokens_details': None, 'prompt_time': 0.018389899, 'prompt_tokens_details': None, 'queue_time': 0.053136704, 'total_time': 0.07287509}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c1354-3f4e-7eb3-8895-ef72ca1f4adb-0', tool_calls=[{'name': 'anime_retriever_tool', 'args': {'query': 'anime similar to Naruto'}, 'id': 'hng4af11n', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 335, 'output_tok

In [31]:
final_response = model_with_tools.invoke(messages)
print(final_response.text)

Why did Naruto go to the party? Because he heard it was a "nin-ja" good time! If you like Naruto, you might enjoy other anime shows like One Piece, Bleach, or Fairy Tail, which also feature action-packed adventures and coming-of-age stories with a similar blend of humor, drama, and fantasy elements.
